<div style='background-image: url("../share/Aerial_view_LLNL.jpg") ; padding: 0px ; background-size: cover ; border-radius: 15px ; height: 250px; background-position: 0% 80%'>
    <div style="float: center ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.8) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.9) ; line-height: 100%">Notebook 2:</div>
            <div style="font-size: x-large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">Visualizing Earthquake Magnitudes: <br><br> Amplitude and Distance</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">2014 South Napa Earthquake.</div>
            <div style="font-size: medium ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">Estimated Time: 30 minutes.</div>
        </div>
    </div>
</div>



This notebook aims to help you explore seismic data resulting from an earthquake.       
Primarily to visualise the idea of amplitude and distance to help make seismic magnitudes more intuitive.       
In the current example we will explore the 2014 South Napa earthquake. Though, you are welcome to play and explore others.

Topics Covered:
- Obtaining Data through ObsPy
- Plotting Earthquake data with Matplotlib
- Querying Event Data

## Visualizing Earthquake Magnitudes Demo

Now that you have learned some Python basics, you can use your skills to analyze some data. 

This notebook will provide some exercises using [Obspy](https://github.com/obspy/obspy/wiki), a Python library used to visualize and analyze seimsographic data.

In [ ]:
%%capture
!pip install --no-cache-dir obspy==1.0.3
!pip install matplotlib==2.0.0

In [3]:
# These import a few functions and set the figure size and plot styles for our visualizations

%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 16, 10


from eps_tools import *

### Event Data

The first step is to find the event. Below, we import the Obspy Client which lets us query for earthquake events. Then we get the UTC date and time for the 2014 South Napa earthquake happened (August 24th, 2014). We are getting the events with at least a magnitude of 6.0. After, we are printing the events in our resulting catalog, with a local projection centered on the San Francisco Bay Area.

In [ ]:
import obspy
from obspy.clients.fdsn import Client

client = Client("USGS")
t = obspy.UTCDateTime("2014-08-24T10:20:44.0")  # South Napa earthquake
cat = client.get_events(starttime=t - 100, endtime=t + 3600,
                        minmagnitude=6)

# Print and plot event.
print(cat)
cat.plot(projection="local", resolution="i");

As you can see above, the map is centered on Santa Rosa, showing a 6.0 magnitude event occuring.

---

### Waveform and Station Data

#### Mass Downloader

ObsPy has a submodule to help download data, called mass downloader. It attempts to offer an API for how seismologists would like to download data. It works in three steps:

1. Define Geographical Domain
2. Define Other Restrictions
3. Launch Download

#### Exercise


##### Part A - Download

Use the mass downloader to download data for a small (there are a lot of stations in this region) geographical region (a circular region with a radius of 1 degree is an example), including the station information, with the mass downloader. Download from 2 minutes before the event to 10 minutes after it and download only `BHZ` (Broadband) channels.

First we'll import the libraries needed to download:

In [ ]:
import obspy
from obspy.clients.fdsn.mass_downloader import CircularDomain, \
    Restrictions, MassDownloader

The cell below will download:

In [ ]:
# download_data is in the eps_tools file

download_data(downloaded = True)

##### Part B - Plot Stations

Plot the stations and the event in a single map. 
The code below reads through the stations file and reads the inventory at each file name. Then the inventory is plotted with a local projection and then a title is added to the map.

In [ ]:
import glob

def plot_stations():
    for i,filename in enumerate(glob.glob("stations/*.xml")):
        if i==0:
            inv = obspy.read_inventory(filename)
        else:
            inv+=obspy.read_inventory(filename)

    fig = inv.plot(projection="local", resolution='i',show=False, size=70)
    fig = cat.plot(fig=fig,projection="local", resolution='i',show=False, title='Stations Available for South Napa Quake');
    plt.show()
    
plot_stations()

##### Part C - Process Data

This will filter the data and check that there are no gaps etc.     
Don't worry about this.

In [13]:
from eps_tools import *

In [18]:
import glob
import os
import numpy as np

# all the stations' data
load_data()

In [15]:
sta = inv.select(station='BKS')[0]
net = sta[0].channels[0]
net.response.get_paz()

Response type: PolesZerosResponseStage, Stage Sequence Number: 1
	From M/S (VELOCITY in Meters per second) to V (VOLTAGE)
	Stage gain: 3503.0, defined at 1.00 Hz
	Transfer function type: LAPLACE (RADIANS/SECOND)
	Normalization factor: 6029.25, Normalization frequency: 1.00 Hz
	Poles: (-0.0124773+0.0122177j), (-0.0124773-0.0122177j), (-34.0006+69.9823j), (-34.0006-69.9823j)
	Zeros: 0j, 0j

In [16]:
st = obspy.read("./waveforms/*.mseed")
st[0].stats.delta

0.02

In [19]:
max_starttime = max(tr.stats.starttime for tr in st)
min_endtime = min(tr.stats.endtime for tr in st)
npts = int((min_endtime - max_starttime)  / 0.04)
st[0].interpolate(sampling_rate=0.5, method="lanczos",
               starttime=max_starttime, npts=npts, a=12)

ValueError: The new array must be fully contained in the old array. No extrapolation can be performed.

In [ ]:
import glob
import os
import numpy as np

inv = obspy.Inventory(networks=[], source="")

# First read all station files.
for filename in glob.glob("./stations/*.xml"):
    inv += obspy.read_inventory(filename)
    
# Now read the waveform files.
st = obspy.read("./waveforms/*.mseed")

# Define Wood-Anderson Response
paz_wa = {'sensitivity': 2800, 'zeros': [0j], 'gain': 1,
          'poles': [-6.2832 - 4.7124j, -6.2832 + 4.7124j]} #https://docs.obspy.org/tutorial/advanced_exercise/advanced_exercise.html

#st.remove_response(inventory=inv, water_level=60, output='DISP') # Remove instrument response

# OR

st.remove_response(inventory=inv, water_level=60) # Remove instrument response
st.simulate(paz_simulate=paz_wa, water_level=60) # Simulate Wood-Anderson 
#st.integrate()  #Integrate to Convert Velocity to Displacemnt [what units?]

#OR

#tr.data = obspy.signal.invsim.estimate_wood_anderson_amplitude()

st.detrend("linear") # Linear Detrend
st.taper(max_percentage=0.05) # Taper
#st.filter("bandpass", freqmin=0.001, freqmax=0.1, zerophase=True, corners=6) # Filter Response

max_starttime = max(tr.stats.starttime for tr in st)
min_endtime = min(tr.stats.endtime for tr in st)
npts = int((min_endtime - max_starttime)  / 0.04)

for tr in st:
    tr.data = np.require(tr.data, requirements=["C_CONTIGUOUS"]) 
    tr.data = tr.data*1000

# Interpolate Data
#st.interpolate(sampling_rate=0.5, method="lanczos", starttime=max_starttime, npts=npts, a=12)

In [20]:
st.select(station='BKS')[0].stats

         network: BK
         station: BKS
        location: 00
         channel: BHZ
       starttime: 2014-08-24T10:20:14.069500Z
         endtime: 2014-08-24T10:25:44.069500Z
   sampling_rate: 40.0
           delta: 0.025
            npts: 13201
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'byteorder': '>', 'dataquality': 'D', 'filesize': 45056, 'encoding': 'STEIM1', 'number_of_records': 11, 'record_length': 4096})

##### Part D - Choose stations and compare amplitudes

**Exercise:** Pick three stations from the above plot.    

       
Try stations:     
1) Roughly in a line and at different distances       
2) Not in a line at the same distance

In [ ]:
# Modify the station labels here before running the cell below this one
StationA = 'BKS'
StationB = 'DBO'
StationC = 'MOD'

Stations = [StationA,StationB,StationC]

In [ ]:
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(10, 8))
outer = gridspec.GridSpec(2, 2, wspace=0.2, hspace=0.2)

for i in range(4):
    inner = gridspec.GridSpecFromSubplotSpec(2, 1,
                    subplot_spec=outer[i], wspace=0.1, hspace=0.1)

In [ ]:
plot_traces(Stations)

In [ ]:
# This is for showing the traces separately
fig,axes = plt.subplots(3,1, sharex=True, sharey=True)
Stations = [StationA,StationB,StationC]

# find max amplitude
max_vals = []
for station in Stations:
    max_vals.append( max(abs(st.select(station=station)[0].data)) )
ylims = [-max(max_vals)*1.1, max(max_vals)*1.1]

for i, station in enumerate(Stations): 
    data = st.select(station=station)[0].data # unnormalized data
    color = colors[i]
    trace = axes[i].plot(st.select(station=station)[0].times(),data, label=station, alpha =0.6, color=color)
    axes[i].axhline(y=max(data), xmin=min(st.select(station=station)[0].times()), xmax=max(st.select(station=station)[0].times()), ls=':', c=color, alpha=0.7)
    axes[i].axhline(y=min(data), xmin=min(st.select(station=station)[0].times()), xmax=max(st.select(station=station)[0].times()), ls=':', c=color, alpha=0.7)
    axes[i].set_ylabel('Amplitude [mm]')
    axes[i].set_ylim(ylims)
    axes[i].set_title(station)

fig.suptitle('Amplitude Comparison')
axes[2].set_xlabel('Time [s]')

# Calculate Amplitude, Distance and Magnitude

## Amplitudes:

Here we calculate print the maximum amplitude at each station.     
Just using the up-down (vertical) component of motion here.

In [ ]:
amplitudes = []

for station in Stations:
    amplitude = max(abs(st.select(station=station)[0].data))
    amplitudes.append(amplitude)
    
print('Ampltiudes: \n\n \
        Station A - {} mm\n \
        Station B - {} mm\n \
        Station C - {} mm\n'.format(amplitudes[0], amplitudes[1], amplitudes[2]))


## Distances:

In [ ]:
def get_stat_lat_long(station):
    stats = st.select(station=station)[0].stats
    ID = '{}.{}.{}.{}'.format(stats.network, stats.station, stats.location, stats.channel)
    lat = inv.get_coordinates(seed_id=ID)['latitude']
    long = inv.get_coordinates(seed_id=ID)['longitude']
    return lat, long

In [ ]:
def calculate_epicentral_distance(event_lat, event_lon, sta_lat, sta_lon):
    from obspy.geodetics import gps2dist_azimuth

    epi_dist, az, baz = gps2dist_azimuth(event_lat, event_lon, sta_lat, sta_lon)
    epi_dist = epi_dist / 1000 # Convert to km
    
    return epi_dist

In [ ]:
event_lat = cat[0].__dict__['origins'][0]['latitude']
event_lon = cat[0].__dict__['origins'][0]['longitude']

epi_dists = []

for station in Stations:
    lat, long = get_stat_lat_long(station)
    epi_dist = calculate_epicentral_distance(event_lat, event_lon, lat, long)
    epi_dists.append(epi_dist)

print('Epicentral Distances: \n\n \
        Station A - {} km\n \
        Station B - {} km\n \
        Station C - {} km\n'.format(epi_dists[0], epi_dists[1], epi_dists[2]))

In [23]:
# I encapsulated the above code into a function that shows distances and amplitudes of the stations
calculate_distance_amplitude()

NameError: name 'cat' is not defined

## All Together

In [ ]:
import pandas as pd

results = pd.DataFrame(
    {'Station': Stations,
     'Distance [km]': epi_dists,
     'Amplitude [mm]': amplitudes
    })

results[['Station', 'Distance [km]', 'Amplitude [mm]']]

![title](fig1.jpg)

Using the figure above, estimate the magnitude.

Put your guess of the magnitude here: [replace_me]

Notebook developed by Alexander Robson, George Khalilieh

Data Science Modules: http:/data.berkeley.edu/education/modules

---
***Please fill out our [feedback form](https://goo.gl/forms/s80KnMAWoozWoS2y1)! Thanks!***